In [1]:
library(tidyverse)
library(RPostgreSQL)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")
list.files(sql_dir)

[1] "basics.sql"              "cohort.sql"             
[3] "icd9.sql"                "lab_tests.sql"          
[5] "lab_unpivot.sql"         "merge_data.sql"         
[7] "population.sql"          "vital_signs.sql"        
[9] "vital_signs_unpivot.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
labs <- "select distinct(label) from vital_signs" %>%
    dbGetQuery(con, .) %>%
    pull(label)
labs

[1] "temp"       "heart_rate" "map"        "cvp"

In [6]:
sql_template <- "
with summary as (
    select distinct icustay_id, label
    , first_value(valuenum) over (partition by icustay_id, label order by charttime) as fst_val
    , first_value(valuenum) over (partition by icustay_id, label order by valuenum) as min_val
    , first_value(valuenum) over (partition by icustay_id, label order by valuenum desc) as max_val
    from vital_signs
)

select icustay_id
, %s
from summary
group by icustay_id
"

In [7]:
sql <- c("max(case when label = '%1$s' then 1 else 0 end) as vs_%1$s_flag",
  "max(case when label = '%1$s' then fst_val else null end) as vs_%1$s_first",
  "max(case when label = '%1$s' then min_val else null end) as vs_%1$s_min",
  "max(case when label = '%1$s' then max_val else null end) as vs_%1$s_max") %>%
paste(collapse = "\n, ") %>%
sprintf(labs) %>%
paste(collapse = "\n, ") %>%
sprintf(sql_template, .)
cat(sql, file = file.path(sql_dir, "vital_signs_unpivot.sql"))

In [8]:
sql %>% make_view_sql("vital_signs_unpivot", "materialized view") %>% dbSendQuery(con, .)

<PostgreSQLResult>

In [9]:
dbGetQuery(con, "select * from vital_signs_unpivot limit 10")

icustay_id,vs_temp_flag,vs_temp_first,vs_temp_min,vs_temp_max,vs_heart_rate_flag,vs_heart_rate_first,vs_heart_rate_min,vs_heart_rate_max,vs_map_flag,vs_map_first,vs_map_min,vs_map_max,vs_cvp_flag,vs_cvp_first,vs_cvp_min,vs_cvp_max
239961,1,36.61111,35.61111,36.66667,1,98,85,112,1,62.000,43.0000,108.000,1,14,14,16
234139,1,35.83333,35.83333,37.83333,1,75,65,92,1,96.000,61.0000,111.000,0,NA,NA,NA
218132,1,37.38889,35.83333,37.38889,1,139,103,139,1,97.000,74.0000,107.000,0,NA,NA,NA
212412,1,35.88889,35.11111,36.88889,1,81,75,90,1,75.000,58.0000,76.000,0,NA,NA,NA
203155,1,38.11111,37.55556,38.38889,1,112,104,127,1,81.000,81.0000,296.000,0,NA,NA,NA
204856,1,36.61111,36.11111,39.11111,1,98,76,115,1,83.000,61.0000,101.000,0,NA,NA,NA
216831,1,36.61111,35.72222,37.66667,1,125,83,130,1,123.667,67.6667,123.667,0,NA,NA,NA
264733,1,38.83333,36.44444,38.83333,1,123,97,130,1,71.000,40.0000,85.000,0,NA,NA,NA
297273,1,36.38889,36.16667,37.33333,1,105,99,130,1,69.000,59.0000,105.000,1,11,10,11
263612,1,36.44444,36.33333,37.94444,1,67,63,87,1,52.000,52.0000,83.000,0,NA,NA,NA


In [10]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE